In [19]:
from qdrant_client import QdrantClient, models
from langchain.embeddings import OpenAIEmbeddings
from dotenv import load_dotenv
load_dotenv()
embeddings = OpenAIEmbeddings()

In [7]:
# create a cient that will store all vector embeddings locally
client = QdrantClient(path="./multiple_vectors/")


In [8]:
client.create_collection(collection_name="multivec", vectors_config={"text": models.VectorParams(size=1536, distance=models.Distance.COSINE),
                                                                     "question": models.VectorParams(size=1536, distance=models.Distance.COSINE)})

True

In [9]:
text1 = "Toronto is the most populous city in Canada and the capital city of the Canadian province of Ontario. With a recorded population of 2,794,356 in 2021, it is the fourth-most populous city in North America. The city is the anchor of the Golden Horseshoe, an urban agglomeration of 9,765,188 people (as of 2021) surrounding the western end of Lake Ontario, while the Greater Toronto Area proper had a 2021 population of 6,712,341. Toronto is an international centre of business, finance, arts, sports and culture, and is recognized as one of the most multicultural and cosmopolitan cities in the world."
question1 = "what is the population of Toronto?"

text2 = "New York's high rate of public transit use, more than 200,000 daily cyclists as of 2014, and many pedestrian commuters make it the most energy-efficient major city in the United States. Walk and bicycle modes of travel account for '21%' of all modes for trips in the city; nationally the rate for metro regions is about 8%. In both its 2011 and 2015 rankings, Walk Score named New York City the most walkable large city in the United States, and in 2018, Stacker ranked New York the most walkable U.S. city. Citibank sponsored public bicycles for the city's bike-share project, which became known as Citi Bike, in 2013. New York City"
question2= "how about public transit in New York?"

In [20]:
client.upsert(collection_name="multivec", points=[
    models.PointStruct(id=1, vector={"text": embeddings.embed_query(text1) , "question": embeddings.embed_query(question1)}), 
    models.PointStruct(id=2, vector={"text": embeddings.embed_query(text2) , "question": embeddings.embed_query(question2)})]
    )

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [21]:
text3 = """
The 2020 United States census showed San Francisco's population to be 873,965, an increase of '8.5%' from the 2010 census. With roughly one-quarter the population density of Manhattan, San Francisco is the second-most densely populated large American city, behind only New York City among cities greater than 200,000 population, and the fifth-most densely populated U.S. county, following only four of the five New York City boroughs.  
San Francisco is part of the five-county San Francisco–Oakland–Hayward, CA Metropolitan Statistical Area, a region of 4.7 million people (13th most populous in the U.S.), and has served as its traditional demographic focal point. It is also part of the greater 14-county San Jose-San Francisco-Oakland, CA Combined Statistical Area, whose population is over 9.6 million, making it the fifth-largest in the United States as of 2018.
"""
text4 = """
The city continues to grow and attract immigrants. A 2019 study by Toronto Metropolitan University (then known as Ryerson University) showed that Toronto was the fastest-growing city in North America. The city added 77,435 people between July 2017 and July 2018. The Toronto metropolitan area was the second-fastest-growing metropolitan area in North America, adding 125,298 persons, compared with 131,767 in the Dallas–Fort Worth–Arlington metroplex in Texas. The large growth in the Toronto metropolitan area is attributed to international migration to Toronto.  
The COVID-19 pandemic in Canada first occurred in Toronto and is among the hotspots in the country.
"""
text5 = """
The city's foreign-born persons made up 47 per cent of the population, compared to 49.9 per cent in 2006. According to the United Nations Development Programme, Toronto has the second-highest percentage of constant foreign-born population among world cities, after Miami, Florida. While Miami's foreign-born population has traditionally consisted primarily of Cubans and other Latin Americans, no single nationality or culture dominates Toronto's immigrant population, placing it among the most diverse cities in the world. In 2010, it was estimated over 100,000 immigrants arrive in the Greater Toronto Area each year.
"""
text6= """
Broadway theatre is one of the premier forms of English-language theatre in the world, named after Broadway, the major thoroughfare that crosses Times Square, sometimes referred to as "The Great White Way". Forty-one venues in Midtown Manhattan\'s Theatre District, each with at least 500 seats, are classified as Broadway theatres. According to The Broadway League, Broadway shows sold approximately $1.27 billion worth of tickets in the 2013–2014 season, an '11.4%' increase from $1.139 billion in the 2012–2013 season. Attendance in 2013–2014 stood at 12.21 million, representing a '5.5%' increase from the 2012–2013 season's 11.57 million. Performance artists displaying diverse skills are ubiquitous on the streets of Manhattan.
"""
text_list = [text3, text4, text5, text6]

In [27]:
for index, item in enumerate(text_list):
    client.upsert(collection_name="multivec", points=[
    models.PointStruct(id=index+3, vector={"text": embeddings.embed_query(item)})]
    )

In [37]:
relevent_list =client.search(collection_name="multivec", query_vector=("question",embeddings.embed_query("how many people in Toronto")), limit=3)
for i in relevent_list:
    print(client.retrieve(collection_name="multivec", ids=[i.id]))

[Record(id=1, payload={}, vector=None)]
[Record(id=2, payload={}, vector=None)]
